In [23]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing_extensions import TypedDict, Annotated
from langgraph.graph import START, StateGraph, END
from langchain_community.document_loaders import WebBaseLoader


In [24]:
class SharedState(TypedDict):
    github_handle: str
    github_profile_content: str
    github_profile_summary: str
    model: ChatOpenAI

In [17]:
def build_model(shared_state: SharedState) -> SharedState:
    model = ChatOpenAI(model= "gpt-5")
    shared_state['model'] = model

    return shared_state

In [26]:
def pull_github_profile_content(shared_state: SharedState) -> SharedState:
    '''Pulls the GitHub profile content for the given GitHub handle.'''
    print('Pulling GitHub profile content...')
    github_profile_url = 'https://www.github.com/' + shared_state['github_handle']
    documents = WebBaseLoader(github_profile_url).load()
    page_content = ''

    for document in documents:
        page_content += document.page_content
    
    shared_state['github_profile_content'] = page_content.strip()
    return shared_state

In [27]:
def summarize_github_profile(shared_state: SharedState) -> SharedState:
    '''Summarizes the GitHub profile content.'''
    print('Summarizing GitHub profile content...')
    model = shared_state['model']
    github_profile_content = shared_state['github_profile_content']

    prompt = f'''Summarize the following GitHub profile content in a concise manner
    and highlight name, organization, followers, location, contact information key skills, 
    projects, and contributions.

    Github Profile Content: {github_profile_content}
    '''
    response = model.invoke(prompt)
    shared_state['github_profile_summary'] = response.content.strip()

    return shared_state


In [28]:
def build_serial_graph():
  # Building a Graph
  # State of the Graph that will be shared among nodes.
  workflow = StateGraph(SharedState)

  # Add nodes.
  workflow.add_node("build_model", build_model)
  workflow.add_node("pull_github_profile_content", pull_github_profile_content)
  workflow.add_node("summarize_github_profile", summarize_github_profile)

  # Define the edges of the graph.
  workflow.add_edge(START, "build_model")
  workflow.add_edge("build_model", "pull_github_profile_content")
  workflow.add_edge("pull_github_profile_content", "summarize_github_profile")
  workflow.add_edge("summarize_github_profile", END)

  graph = workflow.compile()
  
  return graph

In [29]:
load_dotenv()

compiled_graph = build_serial_graph()
response = compiled_graph.invoke({
    'github_handle': 'SauravP97'
})

print(response['github_profile_summary'])

Pulling GitHub profile content...
Summarizing GitHub profile content...
- Name: Saurav Prateek (username: SauravP97)
- Organization/Role: Web Solutions Engineer II at Google (gTech); ex-Software Engineer at GeeksForGeeks
- Followers: 163 (following 10)
- Location: Gurugram, India
- Contact: LinkedIn – https://www.linkedin.com/in/saurav-prateek-7b2096140/
- Tagline: “Practicing the subtle art of not giving a bug!”
- Key skills and interests: Web solutions engineering; Java; TypeScript; Python/Jupyter; AI/ML; LLMs; LangChain; RAG; agentic workflows; distributed systems; low-level design; data structures & algorithms; content/community building
- Notable projects (pinned):
  - Saurav-Low-Level-Design-Template (Java) — 121★, 35 forks
  - Saurav-s-DSA-Templates (Java) — 51★, 31 forks
  - LangPost (TypeScript) — 39★, 4 forks — Agent that summarizes LinkedIn articles into post content
  - AI-Engineering-101 (Jupyter Notebook) — 29★, 4 forks — Collection of agentic workflows
  - micrograd-java

In [ ]:
print(build_serial_graph().get_graph().draw_mermaid())